# Implementación del algoritmo Apriori

##### Por: Daniela Flores Villanueva

En este *notebook* se define lo necesario para implementar el algoritmo Apriori con la menor dependencia de código externo posible. Posteriormente, se emplea la implementación propuesta para minar reglas de asociación que presenten altos valores para ciertas métricas y se analizan separadamente. Finalmente, se entrega una forma de visualizar un conjunto de reglas de asociación.

## Librerías utilizadas

- `numpy`: se utilizó para cargar los datos del Spotify RecSys Challenge, que venían en una arreglo serializado de `numpy`.
- `collections`: se usó su clase `Counter`, que permite obtener un diccionario con la frecuencia de ocurrencia de todos los elementos de una lista.
- `itertools`: se empleó su método `combinations`, para generar todos los subconjuntos de largo `k` que pudiesen extraerse de un conjunto.

In [1]:
import numpy as np
import collections
import itertools

## Clase Apriori

La implementación presentada en este *notebook* se basa en la creación de un objeto de clase `Apriori` que, en primer lugar, recibe los siguientes parámetros:

- `data`: corresponde a las transacciones, en forma de `numpy.array`.
- `min_support`: soporte mínimo para que un `itemset` sea considerado frecuente.
- `min_confidence`: confianza mínima para que cierta regla de asociación sea retornada.
- `min_lift`: *lift* mínimo para que cierta regla de asociación sea retornada.

### Métodos

- `prepare_data`: convierte los datos recibidos (`self.data`) en una lista de conjuntos, donde cada conjunto corresponde a una lista de reproducción. De esta forma, se evitan canciones repetidas en las listas. También, se guarda en `self.songs_counter` un contador con la frecuencia de cada canción. Esta información permitirá formar el conjunto L_1.
- `get_songs_appearances`: gracias a este método, se guarda un diccionario cuyas llaves son nombres de canciones y los valores, el conjunto de índices de las listas de reproducción donde aparece dicha canción. Esto permitirá obtener el contador de soporte de un *itemset* con facilidad.
- `generate_L_1`: con esto se obtienen los *itemsets* frecuentes de largo 1, es decir, las canciones que aparecen un porcentaje de veces mayor al mínimo soporte definido.
- `generate_new_candidates`: este método permite generar un nuevo conjunto de posibles *itemsets* frecuentes. Esto se hace a partir de la unión de un *itemset* con una sola canción. El *itemset* generado debe cumplir con la propiedad de monotonicidad: si un *itemset* es frecuente, sus subconjuntos también lo son. Así, en esta implementación se verifica que todos los subconjuntos de un *itemset* candidato de largo $k$ sean frecuentes en el conjunto $L_{k-1}$ de *itemsets* frecuentes.
- `calculate_subset_count`: gracias al diccionario generado con `get_songs_appearances` es fácil obtener el contador de soporte de un *itemset*: basta con obtener las listas de reproducción en que aparece cada canción del *itemset* y luego intersectar esos conjuntos de *playlists*. El largo de la intersección corresponderá al contador de soporte de *itemset*.
- `prune_itemsets`: retorna el conjunto de *itemsets* que superan el mínimo soporte definido.
- `fit`: método que genera los *itemsets* frecuentes.
- `get_association_rule_from_itemset`: Dado un *itemset* $I$ se forman todas las combinaciones posibles de $X \rightarrow Y$, donde $X \subset I$ y $Y = I - X$. Es importante notar que solo se consideran reglas de asociación aquellas en que tanto X como Y tengan largo mayor o igual a 1.
- `generate`: se generan todas las reglas de asociación posibles a partir de los *itemsets* frecuentes encontrados.
- `get_n_top_rules`: permite obtener las $n$ reglas con mayor valor de la métrica determinada por el usuario. Esta métrica puede corresponder a *confidence* o a *lift*.

In [2]:
class Apriori:
    def __init__(self, data, min_support=0.01, min_confidence=0.01,
                 min_lift=1):
        self.data = data
        self.min_support = min_support
        self.min_confidence = min_confidence
        self.min_lift = min_lift

    def prepare_data(self):
        self.playlists = list(self.data.item().values())
        self.playlists = [set(playlist) for playlist in self.playlists]
        unique_songs = [item for sublist in self.playlists for item in sublist]
        self.songs_counter = collections.Counter(unique_songs)

    def get_songs_appearances(self):
        songs_in_playlists = collections.defaultdict(set)
        for index, playlist in enumerate(self.playlists):
            for song in playlist:
                songs_in_playlists[song].add(index)
        self.songs_in_playlists = songs_in_playlists

    def generate_L_1(self):
        self.L_1_counter = {
            song: times
            for song, times in self.songs_counter.items()
            if times / len(self.playlists) >= self.min_support
        }
        self.L_1 = [{song} for song in self.L_1_counter.keys()]

    def generate_new_candidates(self, current_itemsets, k):
        C_k = set()
        m = k - 2
        for itemset in current_itemsets:
            for song in self.L_1:
                not_prev_frequent = False
                new_candidate = frozenset(itemset).union(song)
                for combination in itertools.combinations(itemset, m):
                    frequent_tuple = frozenset(combination).union(song)
                    if frequent_tuple not in current_itemsets:
                        not_prev_frequent = True
                        break
                if not not_prev_frequent and len(new_candidate) == k:
                    C_k.add(new_candidate)

#         for candidate in current_itemsets:
#             for aux_candidate in current_itemsets:
#                 new_candidate = frozenset(candidate).union(aux_candidate)
#                 if len(new_candidate) == k:
#                     C_k.add(new_candidate)
#         print("Candidates length: {}".format(len(C_k)))
        return C_k

    def calculate_subset_count(self, subset):
        playlists_inter = []
        for song in subset:
            playlists_inter.append(self.songs_in_playlists[song])
        return len(set.intersection(*playlists_inter))

    def prune_itemsets(self, C_k):
        C_k_counter = {}
        for candidate in C_k:
            C_k_counter[candidate] = self.calculate_subset_count(candidate)
        L_k_counter = {
            subset: times
            for subset, times in C_k_counter.items()
            if times / len(self.playlists) >= self.min_support
        }
        return L_k_counter

    def fit(self):
        self.prepare_data()
        self.get_songs_appearances()
        self.generate_L_1()
        self.k_frequent_itemsets = {}
        self.frequent_itemsets = []
        k = 2
        current = self.L_1
        while len(current) != 0:
            C_k = self.generate_new_candidates(current, k)
            L_k_counter = self.prune_itemsets(C_k)
            L_k = L_k_counter.keys()
            self.frequent_itemsets.extend(L_k)
            self.k_frequent_itemsets[k] = sorted(
                L_k_counter.items(), key=lambda x: x[1], reverse=True)
            k += 1
            current = L_k


    def get_association_rule_from_itemset(self, itemset):
        itemset_count = self.calculate_subset_count(itemset)
        itemset_support = itemset_count / len(self.playlists)
        for i in range(1, len(itemset) + 1):
            for x_set in itertools.combinations(itemset, i):
                x_set = frozenset(x_set)
                y_set = frozenset(itemset) - x_set
                x_support = self.calculate_subset_count(x_set) / len(
                    self.playlists)
                x_y_support = self.calculate_subset_count(
                    x_set.union(y_set)) / len(self.playlists)
                rule_confidence = x_y_support / x_support
                if len(x_set) > 0 and len(y_set) > 0:
                    y_support = self.calculate_subset_count(y_set) / len(
                        self.playlists)
                    rule_lift = x_y_support / (x_support * y_support)
                    self.rules["{} -> {}".format(x_set, y_set)] = {
                            "confidence": rule_confidence,
                            "lift": rule_lift
                    }
        
        

    def generate(self, order_by="confidence"):
        self.rules = {}
        for itemset in self.frequent_itemsets:
            self.get_association_rule_from_itemset(itemset)
        return sorted(self.rules.items(), key=lambda x: x[1][order_by], reverse=True)
            
    
    def get_n_top_rules(self, n, order_by="confidence"):
        return sorted(self.rules.items(), key=lambda x: x[1][order_by], reverse=True)[0:n]

In [3]:
spotify_data = np.load("spotify.npy")

In [4]:
apriori = Apriori(
    data=spotify_data, min_support=0.01, min_confidence=0.03, min_lift=0.9)

In [5]:
apriori.fit()

In [6]:
apriori.generate()

[("frozenset({'Mask Off', 'DNA.'}) -> frozenset({'HUMBLE.'})",
  {'confidence': 0.9090909090909092, 'lift': 19.550342130987293}),
 ("frozenset({'XO TOUR Llif3', 'DNA.'}) -> frozenset({'HUMBLE.'})",
  {'confidence': 0.864406779661017, 'lift': 18.589393110989615}),
 ("frozenset({'DNA.'}) -> frozenset({'HUMBLE.'})",
  {'confidence': 0.8225108225108225, 'lift': 17.68840478517898}),
 ("frozenset({'Mask Off', 'XO TOUR Llif3'}) -> frozenset({'HUMBLE.'})",
  {'confidence': 0.8036809815950922, 'lift': 17.283461969786927}),
 ("frozenset({'Bounce Back', 'Broccoli (feat. Lil Yachty)'}) -> frozenset({'Bad and Boujee (feat. Lil Uzi Vert)'})",
  {'confidence': 0.7751937984496124, 'lift': 22.469385462307603}),
 ("frozenset({'XO TOUR Llif3', 'Slippery (feat. Gucci Mane)'}) -> frozenset({'HUMBLE.'})",
  {'confidence': 0.7651515151515151, 'lift': 16.454871293580972}),
 ("frozenset({'Tunnel Vision', 'XO TOUR Llif3'}) -> frozenset({'HUMBLE.'})",
  {'confidence': 0.7500000000000001, 'lift': 16.1290322580645

In [10]:
apriori.get_n_top_rules(25, "lift")

[("frozenset({'X (feat. Future)'}) -> frozenset({'No Heart'})",
  {'confidence': 0.5049019607843137, 'lift': 34.11499735029147}),
 ("frozenset({'No Heart'}) -> frozenset({'X (feat. Future)'})",
  {'confidence': 0.6959459459459459, 'lift': 34.11499735029147}),
 ("frozenset({'Knee Deep (feat. Jimmy Buffett)'}) -> frozenset({'Chicken Fried'})",
  {'confidence': 0.6815286624203822, 'lift': 31.84713375796179}),
 ("frozenset({'Chicken Fried'}) -> frozenset({'Knee Deep (feat. Jimmy Buffett)'})",
  {'confidence': 0.5, 'lift': 31.84713375796179}),
 ("frozenset({'Money Longer'}) -> frozenset({'You Was Right'})",
  {'confidence': 0.6030927835051546, 'lift': 31.087256881709003}),
 ("frozenset({'You Was Right'}) -> frozenset({'Money Longer'})",
  {'confidence': 0.6030927835051546, 'lift': 31.087256881709003}),
 ("frozenset({'Bank Account'}) -> frozenset({'Butterfly Effect'})",
  {'confidence': 0.48260869565217396, 'lift': 24.876736889287315}),
 ("frozenset({'Butterfly Effect'}) -> frozenset({'Bank 